In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import json

In [2]:
doc = pd.read_csv('data/ica_data.csv', encoding='ISO-8859-1', error_bad_lines=False, delimiter=';')
doc.head()

,ID,Timestamp,Tweet,Keluhan,Respon,Bukan Keluhan/Respon,Topik Umum,Topik Spesifik,Lokasi,Waktu,Unnamed: 10
0,"6,78845E+17","21 Des 2015, 02.50.25 PM",@EL_Atheos @ridwankamil ya mungkin karena pere...,NaN,NaN,Ya,Bukan Keluhan,-,-,-,NaN
1,"6,78846E+17","21 Des 2015, 02.53.02 PM",@ridwankamil @dbmpkotabdg kang teman saya tert...,Ya,NaN,NaN,Lingkungan Hidup,pohon tumbang,Jalan Sangkuriang depan Polsek Coblong,-,NaN
2,"6,78847E+17","21 Des 2015, 02.57.55 PM",Di tribun jabar biasanya suka di post agenda k...,NaN,NaN,Ya,Bukan Keluhan,-,-,-,NaN
3,"6,78847E+17","21 Des 2015, 02.58.54 PM",@dbmpkotabdg RT @fajriattack: Lapor pak @ridwa...,Ya,NaN,NaN,Infrastruktur,lampu penerangan jalan umum,Jalan depan Kampus LPKIA,-,NaN
4,"6,78849E+17","21 Des 2015, 03.06.49 PM","@diskamtam bapak/ibu mau tanya, kalo pemelihar...",NaN,NaN,Ya,Bukan Keluhan,-,-,-,NaN


In [3]:
data = doc.loc[:, 'Tweet':'Bukan Keluhan/Respon']
data.head()

,Tweet,Keluhan,Respon,Bukan Keluhan/Respon
0,@EL_Atheos @ridwankamil ya mungkin karena pere...,NaN,NaN,Ya
1,@ridwankamil @dbmpkotabdg kang teman saya tert...,Ya,NaN,NaN
2,Di tribun jabar biasanya suka di post agenda k...,NaN,NaN,Ya
3,@dbmpkotabdg RT @fajriattack: Lapor pak @ridwa...,Ya,NaN,NaN
4,"@diskamtam bapak/ibu mau tanya, kalo pemelihar...",NaN,NaN,Ya


In [4]:
data = data.fillna('0')
data['Keluhan'] = data['Keluhan'].map({'Ya':1})
data['Respon'] = data['Respon'].map({'Ya':1})
data['Bukan Keluhan/Respon'] = data['Bukan Keluhan/Respon'].map({'Ya':1})

In [5]:
data_text = data.loc[:, 'Tweet']
data_tweet = data.loc[:, 'Tweet']
data_tweet[0]

u'@EL_Atheos @ridwankamil ya mungkin karena perempuan bj merah yg paling kelihatan karena berada di depan, laki2 jg sering jadi bahan joke RK'

In [6]:
data_keluhan = data.loc[:, 'Keluhan']
data_keluhan[0]

nan

In [7]:
import requests
import re
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [8]:
def removeURL(sent):
    # remove username
    sent = re.sub('@[^\s]+','',sent)
    sent = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', sent, flags=re.MULTILINE)
    return sent

In [9]:
def normalization(sent):
    print(sent)
    # text normalization
    header = {
        'Content-Type' : 'application/json',
        'x-api-key' : 'PGSpcyOiDPqdlNmQ1kd66p59qhTDuLUOCqdJk4sF'
    }
    req = requests.post("https://api.prosa.ai/v1/normals", headers=header, json = { 'text' : sent})
    res = req.json()
    print(res)

In [10]:
def stemmer(sent):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return stemmer.stem(sent)

In [11]:
def tokenizer(sent):
    words = word_tokenize(sent)
    print (words)
    return words

In [12]:
data_temp = []
for tweet in data_tweet:
    tweet = removeURL(tweet)
    data_temp.append(tweet)
data_tweet = data_temp

In [13]:
# data_temp = []
# for tweet in data_tweet:
#     tweet = stemmer(tweet)
#     data_temp.append(tweet)
# data_tweet = data_temp

In [14]:
data_temp = []
for tweet in data_tweet:
    tweet = tokenizer(tweet)
    data_temp.append(tweet)
data_tweet = data_temp

[u'ya', u'mungkin', u'karena', u'perempuan', u'bj', u'merah', u'yg', u'paling', u'kelihatan', u'karena', u'berada', u'di', u'depan', u',', u'laki2', u'jg', u'sering', u'jadi', u'bahan', u'joke', u'RK']
[u'kang', u'teman', u'saya', u'tertimpa', u'pohn', u'dijln', u'sangkuriang', u'dpn', u'polsek', u'coblong', u'tlg', u'ditertibkan', u'phn', u'yg', u'sdh', u'lapuknuhun']
[u'Di', u'tribun', u'jabar', u'biasanya', u'suka', u'di', u'post', u'agenda', u'kang', u'emil', u'.', u'Tapi', u'gak', u'setiap', u'hari', u'sih', u'.', u'RT', u'Klo', u'saya', u'k', u'Bdg', u'bs', u'ketemu', u'Kang', u'dmn']
[u'RT', u'Lapor', u'pak', u'lampu', u'penerangan', u'jalan', u'depan', u'kampus', u'lpkia', u'ngaplek', u':', u'(', u':', u'(', u':', u'(']
[u'bapak/ibu', u'mau', u'tanya', u',', u'kalo', u'pemeliharaan', u'taman2', u'yg', u'banyak', u'dibagun', u'skr', u'gimana', u'nantinya', u'?']
[u'Pa', u'tempat', u'kerja', u'saya', u'tgl', u'25', u'tetep', u'masuk', u'.', u'gimana', u'itu', u'aturannya', u'pa',

[u'Guntoro', u':', u'Pembangunan', u'Fly', u'Over', u'di', u'Bandung', u'Ditarget', u'2016']
[u'pak', u'mhn', u"tndkan'y", u'ats', u'prkir', u'liar', u'd', u'jl', u'pasirkaliki', u'ada', u'petugas', u'brjaga', u'pun', u'tdk', u'd', u'tndak', u'lnjuti']
[u'#', u'suaraBDG', u'via', u':', u'parkir', u'liar', u'di', u'jl', u'pasirkaliki', u'ada', u'petugas', u'brjaga', u'pun', u'tdk', u'd', u'tndak', u'lnjuti', u'cc']
[u'RT', u'#', u'suaraBDG', u'via', u':', u'parkir', u'liar', u'di', u'jl', u'pasirkaliki', u'ada', u'petugas', u'brjaga', u'pun', u'tdk', u'd', u'tndak', u'lnjuti', u'cc']
[u'RT', u'#', u'suaraBDG', u'via', u':', u'parkir', u'liar', u'di', u'jl', u'pasirkaliki', u'ada', u'petugas', u'brjaga', u'pun', u'tdk', u'd', u'tndak', u'lnjuti', u'cc']
[u'RT', u'#', u'suaraBDG', u'via', u':', u'parkir', u'liar', u'di', u'jl', u'pasirkaliki', u'ada', u'petugas', u'brjaga', u'pun', u'tdk', u'd', u'tndak', u'lnjuti', u'cc']
[u'Suka', u'pengen', u'ngegambar', u"'cantik", u"'", u'di', u'mobi

[u'hatur', u'nuhun', u'dbmp', u'lampu', u'jln', u'merdeka', u'parantos', u'sae', u'indah', u'pisan', u'disarengan', u'ku', u'cai', u'solokan', u'nu', u'ngalir', u'bersih']
[u'RT', u'hatur', u'nuhun', u'dbmp', u'lampu', u'jln', u'merdeka', u'parantos', u'sae', u'indah', u'pisan', u'disarengan', u'ku', u'cai', u'solokan', u'nu', u'ngalir', u'bersih', u'h']
[u'Kang', u'di', u'instagram', u':', u'Pengumuman', u':', u'1', u'.', u'Dilarang', u'masuk', u'ke', u'Teras', u'Cikapundung', u'jln', u'Siliwangi', u'krn', u'belum', u'buka', u'unt']
[u'tetapi', u'hari', u'ini', u'masih', u'boleh', u'akan', u'terjadi', u'pada', u'hari', u'esok', u'.']
[u'pak', u'emil', u',', u'PJU', u'anu', u'di', u'jl.pahlawan', u'deket', u'kampus', u'LP31', u'mati', u'wae', u'pak', u',', u'gelap', u'.', u'Rawan', u'kecelakaan/begal..', u'Haturnuhun']
[u'RT', u'Kang', u'di', u'instagram', u':', u'Pengumuman', u':', u'1', u'.', u'Dilarang', u'masuk', u'ke', u'Teras', u'Cikapundung', u'jln', u'Siliwangi', u'krn', u'belu

[u'RT', u'Dari', u'sukajadi', u'sampai', u'lembang', u'macet', u'total+tak', u'ada', u'polisi', u'yg', u'atur', u'?', u'?', u'#', u'lalinBDG']
[u'Bukan', u'di', u'Tebing', u'Keraton', u'tapi', u'ini', u'masih', u'di', u'Jawa', u'Barat', u'euy', u'.', u'#', u'JabarKahiji', u'?']
[u'Bukan', u'di', u'Tebing', u'Keraton', u'tapi', u'ini', u'masih', u'di', u'Jawa', u'Barat', u'euy', u'.', u'#', u'JabarKahiji', u'?']
[u'Perilaku', u'melanggar', u'rambu', u'lalu', u'lintas', u'di', u'jalan', u'Purnawarman', u'perlu', u'di', u'tindak', u'tegas']
[u'terimakasih', u'pak', u',', u'skr', u'sampah', u'sdg', u'di', u'angkut']
[u'13.32', u':', u'RT', u'Penertiban', u'PKL', u'di', u'Jalan', u'Trunojoyo', u',', u'Jalan', u'Aceh', u'&', u'amp', u';', u'Jl', u'Diponegoro', u'(', u'23/12/2015', u')', u'.']
[u'RT', u'Penertiban', u'pkl', u'di', u'jl', u'.', u'Trunojoyo', u',', u'jl', u'.', u'Aceh', u'&', u'amp', u';', u'jl', u'.', u'Diponegoro', u'23/12/2015', u'cc']
[u'RT', u'Penertiban', u'pkl', u'di', u

[u'Punten', u'ieu', u'lampu', u'PJU', u'di', u'jl.pungkur', u'(', u'dekat', u'hotel', u'brawijaya', u')', u'pareum', u'.', u'Hatur', u'nuhun']
[u'kang', u'rencana', u'buat', u'sarana', u'kesehatan', u'di', u'bandung', u'udh', u'ada', u'?', u'kan', u'taman', u'udh', u'banyak', u'di', u'bangun', u'tinggal', u'kesehatannya', u'kapan', u'?']
[u'kang', u'rencana', u'buat', u'sarana', u'kesehatan', u'di', u'bandung', u'udh', u'ada', u'?', u'kan', u'taman', u'udh', u'banyak', u'di', u'bangun', u'tinggal', u'kesehatannya', u'.kapan', u'?']
[u'kata', u'kang', u'di', u'bandung', u'klo', u'malem', u'minggu', u'suka', u'ada', u'appolinaris', u'night', u'.', u'Banyak', u'stan', u'makanan', u'di', u'sana']
[u'punten', u'ah', u'bade', u'laporan', u'naha', u'nya', u'asa', u'aneh', u'ari', u'catatan', u'sipil', u'meni', u'sering', u'pisan', u'salah', u'nyerat', u'nami', u',', u'abdi', u'jadi', u'sesah', u'ngadamel', u'paspor']
[u'Pak', u'pami', u'rajia', u'tempat', u'sampah', u'msh', u'ada', u'?', u'?'

...

[u'masih', u'gak', u'rela', u'kalau', u'rk', u'ninggalin', u'bdg', u'.', u'Kita', u'org', u'bdg', u'br', u'bbrp', u'thn', u'ini', u'merasa', u'benar2', u'diurusi', u'.', u'Do', u"n't", u'leave', u'bdg', u'please']
[u'RT', u'14.20', u':', u'RT', u'Pelanggaran', u'rambu', u'S', u'coret', u'di', u'Jln', u'Supratman', u'tadi', u'siang', u'.', u'cc']
[u'RT', u'Kata', u'soal', u'kisruh', u'LGBT', u':', u'Seharusnya', u'yang', u'dibangun', u'dialog', u'bukan', u'provokasi', u'.']
[u'RT', u'Kata', u'soal', u'kisruh', u'LGBT', u':', u'Seharusnya', u'yang', u'dibangun', u'dialog', u'bukan', u'provokasi', u'.']
[u'RT', u'Kata', u'soal', u'kisruh', u'LGBT', u':', u'Seharusnya', u'yang', u'dibangun', u'dialog', u'bukan', u'provokasi', u'.']
[u'RT', u'Kata', u'soal', u'kisruh', u'LGBT', u':', u'Seharusnya', u'yang', u'dibangun', u'dialog', u'bukan', u'provokasi', u'.']
[u'RT', u'Penertiban', u'PKL', u'di', u'sepanjang', u'jl.RE.Martadinata']
[u'Lagian', u'LGBT', u'itu', u'kaya', u'nolak', u'takdir',

[u'RT', u'nanti', u'tersambung', u'dgn', u'LRT', u'Bandung', u'Raya', u'.', u'*apalagi', u'ditemeni', u'gebetan', u',', u'wuss', u'tdk', u'akan', u'kerasa', u'perjalanannya', u'.']
[u'RT', u'Pak', u'jika', u'berkenan', u'bapa', u'membangun', u'Bandung', u',', u'biar', u'Pak', u'membangun', u'Jakarta', u'lagi', u'.', u'Betul', u'pak', u'?']
[u'Ngadu', u'ke', u'siapa', u'di', u'margacinta', u'byk', u'rentenir', u'yg', u'atasnamakan', u'koprasi', u'smpan', u'pnjam..mresahkan', u'rmah', u'tangga', u'aja']
[u'RT', u'semua', u'yang', u'anti', u'Pancasila', u'juga', u'apapun', u'itu', u'tentunya', u'akan', u'diajak', u'dialog', u'bkn', u'frontal', u'.', u'tidak', u'perlu', u'sebut', u'nama', u',', u'krn', u'byk', u'.', u'https', u':']
[u'Pa', u'wali', u'yg', u'tercinta', u',', u'di', u'berita', u'byk', u'problem', u'sampah', u',', u'apakah', u'ada', u'rencana', u'buat', u'bikin', u'BANK', u'SAMPAH', u'di', u'Bandung', u'#', u'BandungBersih']
[u'RT', u'Kata', u'soal', u'kisruh', u'LGBT', u':',

[u'RT', u'semua', u'yang', u'anti', u'Pancasila', u'juga', u'apapun', u'itu', u'tentunya', u'akan', u'diajak', u'dialog', u'bkn', u'frontal', u'.', u'tidak', u'perlu', u'sebut', u'nama', u',', u'krn', u'byk', u'.', u'https', u':']
[u'RT', u'#', u'lalinBDG', u'18.16', u':', u'Jl', u'Moch', u'Toha', u'Banjir', u'cc', u':', u'|']
[u'RT', u'#', u'suaraBDG', u'via', u'Tombol', u'sama', u'lampu', u'penyebrangannya', u'engga', u'berfungsi', u'nih', u',', u'pak', u'cc', u':']
[u'RT', u'18.49', u'Via', u'URC', u'Tegallega', u',', u'banjir', u'jl', u'Leuwipanjang', u'surut']
[u'RT', u'semua', u'yang', u'anti', u'Pancasila', u'juga', u'apapun', u'itu', u'tentunya', u'akan', u'diajak', u'dialog', u'bkn', u'frontal', u'.', u'tidak', u'perlu', u'sebut', u'nama', u',', u'krn', u'byk', u'.', u'https', u':']
[u'cuma', u'yg', u'jadi', u'kendala', u':', u'transport', u'di', u'Bandung', u'msh', u'kurang', u'memadai', u'.', u'semoga', u'bisa', u'adu', u'cepat', u'utk', u'membereskan', u'transportasi', u'di

[u'Kang', u'info', u'dr', u'tmn', u'Dealer', u'Honda', u'A', u'Yani', u',', u'nenek', u'kbingungan', u'di', u'jl', u'Garuda', u'sblh', u'gereja', u'.', u'Info', u'Budi', u'087825016444']
[u'Ini', u'sama', u'aja', u'dgn', u'heboh', u'rambu', u'S', u'dan', u'P.', u'Kendaraan', u'Polisi', u'bebas', u'Berhenti', u'dan', u'Parkir', u'di', u'rambu', u'tsb', u',', u'warga', u'di', u'tilang', u'.']
[u'hatur', u'nuhun', u'atos', u'nyla', u'#', u'bandungjuara', u',', u'kalau', u'bs', u'ad', u'tempat', u'sampah', u'di', u'depan', u'gudeg', u'banda', u'upami', u'sok', u'kotor', u'pak']
[u'RT', u'Penertiban', u'penjual', u'laser', u'di', u'pinggir', u'jl.Cicadas', u'&', u'amp', u';', u'jl.Jakarta', u'cc', u'https', u':']
[u'ass.wr.wb', u'pak', u'punten', u'upami', u'di', u'jalan', u'depan', u'r.m', u'.', u'gudeg', u'banda', u'teh', u'sok', u'kotor', u',', u'tiasa', u'sadiakeun', u'tempat', u'sampah', u'di', u'depan', u'r.m.nya', u'pak', u'?']
[u'kade', u'Kang', u'sepertinya', u'Haters', u'mulai', u

[u'RT', u'Di', u'Katamso', u'dpn', u'kursus', u'bahasa', u'sptnya', u'ada', u'kebocoran', u'pipa', u'mhn', u'dicek', u'krn', u'berpotensi', u'rusak', u'jln', u'cc']
[u'Waspada', u'terhadap', u'dukungan', u'tdk', u'tulus', u'dr', u'org2', u'yg', u'tdk', u'suka', u'Indonesia', u'maju', u'.', u'Saya', u'dukung', u'Bp', u'utk', u'terus', u'mengabdi', u'di', u'Bandung', u'hingga', u'JB1']
[u'Dapat', u'kami', u'informasikan', u'jika', u'air', u'yang', u'keluar', u'itu', u'berasal', u'dari', u'bak', u'kontrol', u'telkom', u'bukan', u'dari', u'pipa', u'bocor', u'.']
[u'bagus', u'bangunan', u'yg', u'langgar', u'dpt', u'sanksi', u'.', u'Hanya', u'utk', u'lahan', u'pengganti', u'harus', u'dlm', u'satu', u'zona', u'agar', u'fungsi', u'hidrologi', u'dpt', u'terkompensasi', u'.']
[u'ngalonyeng', u'keneh', u',', u'teu', u'kapokkeun', u'geuningan', u'...', u'#', u'TepokJidat']
[u'pa', u'tong', u'nyalon', u'ka', u'jakarta', u'atuh', u'pa', u'warga', u'BDG', u'masih', u'nyaah', u'ka', u'bapa..', u'?', u

In [15]:
y = data_keluhan

In [16]:
model = Word2Vec(data_tweet, min_count=1)
model.train(data_tweet, total_examples=1, epochs=1)
x = model.wv

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
text_train, text_test, y_train, y_test = train_test_split(data_text, y, test_size=0.25, random_state=42)

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

TypeError: Expected sequence or array-like, got <class 'gensim.models.keyedvectors.Word2VecKeyedVectors'>